# Show webcam feed in notebook

Use OpenCV to read webcam feed, use ipywidgets to display videostream.

In [ ]:
import time
import cv2
import threading
from ipywidgets import widgets
from IPython.display import display

In [ ]:
class WebcamThread(threading.Thread):
    """
    Thread to read frame by frame from webcam stream and update the
     interactive image widget.
    """
    def __init__(self, interactive_img):
        """
        Create thread.
        
        Args:
            interactive_img (Widget): Interactive widget to display image
        """
        super(WebcamThread, self).__init__()
        self._stop_event = threading.Event()
        self.interactive_img = interactive_img
        self.camera = cv2.VideoCapture(0)

    def stop(self):
        """
        Stop thread.
        """
        self._stop_event.set()

    def stopped(self):
        """
        True iff tread is stopped.
        """
        return self._stop_event.is_set()
    
    def run(self):
        """
        Start thread.
        Creates loop that reads frame from webcam and updates interactive image
         until thread is stopped.
        """
        is_capturing = self.camera.isOpened()
        while is_capturing and not self.stopped():
            is_capturing, frame = self.camera.read()
            interactive_img.value = cv2.imencode(
                '.png', frame)[1].tostring()
            time.sleep(0.01)
        self.camera.release()

In [ ]:
# Create interactive image
interactive_img = widgets.Image(
    value=b'',
    format='png',
    width=800,
    height=600,
)

# Create thread to update interactive image with webcam
thread = WebcamThread(interactive_img)
display(interactive_img)
thread.start()

# Stop thread upon exit
while True:
    try:
        time.sleep(1)
    except:
        thread.stop()
        thread.join()
        break

print('Finish')